In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import pairwise_distances

In [2]:
data_cols=['userID','movieID','Ratings','Timestamp']
data_rating=pd.read_csv('ml-100k/u.data',sep='\t',names=data_cols,encoding='latin-1')            

In [3]:
rate_pivot=data_rating.pivot_table(index='userID',columns='movieID',values='Ratings')
rate_pivot    
rate_pivot.index = rate_pivot.index + 1

In [4]:
rate_pivot=rate_pivot.replace(np.nan,0)
rate_pivot
rate_pivot.index = pd.Index(range(1, len(rate_pivot) + 1))
print(rate_pivot.index)

RangeIndex(start=1, stop=944, step=1)


In [5]:
user_similarity=pairwise_distances(rate_pivot,metric='cosine')
movie_similarity=pairwise_distances(rate_pivot,metric='cosine')

In [6]:
usersimidf=pd.DataFrame(user_similarity,index=rate_pivot.index,columns=rate_pivot.index)
usersimidf

,1,2,3,4,5,6,7,8,9,10,...,934,935,936,937,938,939,940,941,942,943
1,2.442491e-15,8.330690e-01,0.952540,0.935642,0.621525,0.569761,0.559633,0.680928,0.921862,0.623456,...,0.630473,0.880518,0.725124,0.810295,0.802674,0.881905,6.859280e-01,0.851383,8.204921e-01,0.601825
2,8.330690e-01,3.330669e-16,0.889409,0.821879,0.927021,0.754157,0.892672,0.896656,0.838952,0.840138,...,0.843014,0.692058,0.641211,0.575954,0.680111,0.771417,7.732100e-01,0.838515,8.277322e-01,0.894202
3,9.525405e-01,8.894087e-01,0.000000,0.655849,0.978755,0.927585,0.933863,0.916940,0.938960,0.934849,...,0.968125,0.957247,0.836171,0.930962,0.875755,0.973729,8.381099e-01,0.898757,8.665839e-01,0.973444
4,9.356422e-01,8.218788e-01,0.655849,0.000000,0.968196,0.931956,0.908770,0.811940,0.898716,0.939141,...,0.947893,0.963216,0.866885,0.806529,0.853942,0.969862,8.031418e-01,0.847959,8.299139e-01,0.941248
5,6.215248e-01,9.270210e-01,0.978755,0.968196,0.000000,0.762714,0.626400,0.751070,0.943153,0.798573,...,0.661206,0.919420,0.905076,0.920221,0.851393,0.928541,7.600453e-01,0.860405,8.475026e-01,0.686059
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,8.819047e-01,7.714166e-01,0.973729,0.969862,0.928541,0.888148,0.892973,0.904102,0.960148,0.928540,...,0.933961,0.568846,0.741979,0.773551,0.567334,0.000000,9.123129e-01,0.819971,9.567357e-01,0.855750
940,6.859280e-01,7.732100e-01,0.838110,0.803142,0.760045,0.647551,0.670075,0.753117,0.879505,0.657039,...,0.672847,0.892976,0.812464,0.818683,0.824842,0.912313,3.330669e-16,0.854848,7.386238e-01,0.758972
941,8.513831e-01,8.385152e-01,0.898757,0.847959,0.860405,0.855554,0.940007,0.853855,0.856755,0.909695,...,0.953048,0.796699,0.711682,0.765789,0.686600,0.819971,8.548477e-01,0.000000,8.983582e-01,0.904880
942,8.204921e-01,8.277322e-01,0.866584,0.829914,0.847503,0.682672,0.717997,0.824678,0.907503,0.787670,...,0.773560,0.926487,0.910412,0.870446,0.900615,0.956736,7.386238e-01,0.898358,1.110223e-16,0.817535


In [7]:
def userbased(userid,rate_pivot,usersimidf):
    unrated_movies=rate_pivot.columns[rate_pivot.loc[userid]==0]
    recommend=[]
    similarity_scores=usersimidf[userid]
    for movie in unrated_movies:
        weighted_sum=0
        similarity_sum=0
        for other_user in range(0, len(rate_pivot)):
            user=other_user+1
            if movie in rate_pivot.columns and other_user in rate_pivot.index:
                #calculating average and centaralising or normalising
                avg_rating=rate_pivot.loc[other_user][rate_pivot.loc[other_user] != 0].mean()
                #avg_rating=nonzero_rating.mean()
                center_rating=rate_pivot.at[other_user, movie] - avg_rating
                #calculating weighted sum and similarity sum
                weighted_sum = weighted_sum + similarity_scores[other_user] * center_rating
                similarity_sum = similarity_sum + abs(similarity_scores[other_user])
            if similarity_sum > 0:
                predicted=weighted_sum/similarity_sum
            else:
                predicted=0
            recommend.append((movie,predicted))
    recommend=sorted(recommend,key=lambda x:x[1],reverse=True)    
    print(recommend[:10])
        

In [8]:
moviesimi=pd.DataFrame(movie_similarity,index=rate_pivot.index,columns=rate_pivot.index)

In [9]:
def item_based(userid,rate_pivot,movie_similarity):
    recommend=[]
    similarity_scores=movie_similarity[userid]
    for movie in rate_pivot.loc[userid][rate_pivot.loc[userid]!=0].index:
        weighted_sum=0
        similarity_sum=0
        for other_user in range(1,len(rate_pivot)):
            if movie in rate_pivot.columns and other_user in rate_pivot.index:
                avg_rating=rate_pivot.loc[other_user][rate_pivot.loc[other_user]!=0].mean()
                center_rating=rate_pivot.at[other_user,movie]-avg_rating
                weighted_sum=weighted_sum+similarity_scores[other_user] * center_rating
                similarity_sum=similarity_sum+abs(similarity_scores[other_user])
        if similarity_sum > 0:
            predicted=weighted_sum/similarity_sum
        else:
            predicted=0
        recommend.append((movie,predicted))
    recommend=sorted(recommend,key=lambda x:x[1],reverse=True)    
    print(recommend[:10])               
    

In [11]:
userid=90
userbased(userid,rate_pivot,usersimidf)    

[(1, 1.3897058823529411), (13, 1.3897058823529411), (15, 1.3897058823529411), (16, 1.3897058823529411), (32, 1.3897058823529411), (44, 1.3897058823529411), (48, 1.3897058823529411), (50, 1.3897058823529411), (55, 1.3897058823529411), (81, 1.3897058823529411)]


In [10]:
userid=90
item_based(userid,rate_pivot,moviesimi)  

[(100, -1.4850443176604742), (258, -1.527214825395059), (286, -1.7684075246142308), (127, -1.851499118411746), (300, -1.8795315424653647), (174, -1.8989053262926259), (313, -2.012249326005318), (98, -2.0122558908803403), (56, -2.070448449256507), (237, -2.123142992847115)]
